# Mandatory Exercise - Session 5

### Students: Nafis Banirazi & Jan Carbonell

### Lab Objective:
The Objective of this lab is to categorize the given pairs, print their most frequent WordNet synset, their corresponding least common subsumer (LCS) and their similarity using the following functions:

- Path Similarity
- Leacock-Chodorow Similarity
- Wu-Palmer Similarity
- Lin Similarity


In [1]:
# initial imports. Could also be done in the PC
import nltk
import numpy as np
import pandas as pd
import itertools

#additional set of imports
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic


#given set of pairs
pairs = [('the', 'DT'), ('man', 'NN'), ('swim', 'VB'), \
         ('with', 'PR'), ('a', 'DT'), ('girl', 'NN'), \
         ('and', 'CC'), ('a', 'DT'), ('boy', 'NN'), \
         ('whilst', 'PR'), ('the', 'DT'), ('woman', 'NN'), \
         ('walk', 'VB')]
n = {}
v = {}
aux = {}
freq = []
lcs = []
definition = []

Now, for each pair, we will search for their most frequent WordNet sysnset. In the documentation we can find that there are also adjectives and adverbs listed as options but are not used in this given set of pairs. Listing it as a reference: https://wordnet.princeton.edu/documentation/wn1wn

In [2]:
for e in pairs:
    if e[0] not in n and e[0] not in v:
        if e[1] == 'NN':
            n[e[0]] = ['noun']
            n[e[0]] += [wn.synset(e[0]+'.n.01')]
        elif e[1] == 'VB':
            v[e[0]] = ['verb']
            v[e[0]] += [wn.synset(e[0]+'.v.01')]
            
#verification that it is properly stored
for keys,values in n.items():
    print(keys, values)
    
for keys,values in v.items():
    print(keys, values)
            

man ['noun', Synset('man.n.01')]
girl ['noun', Synset('girl.n.01')]
boy ['noun', Synset('male_child.n.01')]
woman ['noun', Synset('woman.n.01')]
swim ['verb', Synset('swim.v.01')]
walk ['verb', Synset('walk.v.01')]


For each of the pairs that was found to be on WordNet, we now get their corresponding least commmon subsumer. That is the most specific common ancestor (hypernym) of two concepts found in a given ontology. For example, the LCS of moose and kangaroo in WordNet is mammal.

In [3]:
for key in n:
    if key not in freq:
        freq += [(key, n[key][0], n[key][1])]

for key in v:
    if key not in freq:
        freq += [(key, v[key][0], v[key][1])]

print(freq)
print(freq[0][0])

[('man', 'noun', Synset('man.n.01')), ('girl', 'noun', Synset('girl.n.01')), ('boy', 'noun', Synset('male_child.n.01')), ('woman', 'noun', Synset('woman.n.01')), ('swim', 'verb', Synset('swim.v.01')), ('walk', 'verb', Synset('walk.v.01'))]
man


In [5]:
for key in freq:
    row = []
    for alt in freq:
        row.append(key[2].lowest_common_hypernyms(alt[2]))
    lcs.append(row)
    
for row in lcs:
    print(row)
    print(len(row))

[[Synset('man.n.01')], [Synset('adult.n.01')], [Synset('male.n.02')], [Synset('adult.n.01')], [], []]
6
[[Synset('adult.n.01')], [Synset('girl.n.01')], [Synset('person.n.01')], [Synset('woman.n.01')], [], []]
6
[[Synset('male.n.02')], [Synset('person.n.01')], [Synset('male_child.n.01')], [Synset('person.n.01')], [], []]
6
[[Synset('adult.n.01')], [Synset('woman.n.01')], [Synset('person.n.01')], [Synset('woman.n.01')], [], []]
6
[[], [], [], [], [Synset('swim.v.01')], [Synset('travel.v.01')]]
6
[[], [], [], [], [Synset('travel.v.01')], [Synset('walk.v.01')]]
6


In [10]:
print(len(freq))
lcs_np = np.array(lcs)
pd.DataFrame(lcs_np, columns=freq, index=freq)

6


,"(man, noun, Synset('man.n.01'))","(girl, noun, Synset('girl.n.01'))","(boy, noun, Synset('male_child.n.01'))","(woman, noun, Synset('woman.n.01'))","(swim, verb, Synset('swim.v.01'))","(walk, verb, Synset('walk.v.01'))"
"(man, noun, Synset('man.n.01'))",[Synset('man.n.01')],[Synset('adult.n.01')],[Synset('male.n.02')],[Synset('adult.n.01')],[],[]
"(girl, noun, Synset('girl.n.01'))",[Synset('adult.n.01')],[Synset('girl.n.01')],[Synset('person.n.01')],[Synset('woman.n.01')],[],[]
"(boy, noun, Synset('male_child.n.01'))",[Synset('male.n.02')],[Synset('person.n.01')],[Synset('male_child.n.01')],[Synset('person.n.01')],[],[]
"(woman, noun, Synset('woman.n.01'))",[Synset('adult.n.01')],[Synset('woman.n.01')],[Synset('person.n.01')],[Synset('woman.n.01')],[],[]
"(swim, verb, Synset('swim.v.01'))",[],[],[],[],[Synset('swim.v.01')],[Synset('travel.v.01')]
"(walk, verb, Synset('walk.v.01'))",[],[],[],[],[Synset('travel.v.01')],[Synset('walk.v.01')]


We then plot the results, not only to better visualize the specific x and y coordinates that we have obtained and store but also to connect the dots and evaluate the overall increase in performance and benchmarking against the other models. 

In [ ]:
#adding the necessary packages for plotting
import matplotlib.pyplot as plt

#preparing and plotting the accuracy graph
x = train_stop
plt.figure()
plt.plot(x, accuracy['HMM'], label='HMM')
plt.plot(x, accuracy['TnT'], label='TnT')
plt.plot(x, accuracy['PER'], label='PER')
plt.plot(x, accuracy['CRF'], label='CRF')

#adding the legend showing the plot
plt.xlabel('Number of Sentences')
plt.ylabel('Model Accuracy')
plt.title('Part Of Speech Models')
plt.legend()
plt.show()

<b>Which model would you select?</b> Justify the answer.

Based on the initial graph, both CRF and PER seem like the better performing algorithms. If we could only make a decision based on this data, we would pick the Perceptron model.

In order to pick the best overall model, another relevant measure is the speed of the system. Because of this, we also have implemented a timer in between each of the models and will plot it accordingly to figure out which algorithms were more effective. 

In [ ]:
#preparing and plotting the accuracy graph
x = train_stop
plt.figure()
plt.plot(x, time['HMM'], label='HMM')
plt.plot(x, time['TnT'], label='TnT')
plt.plot(x, time['PER'], label='PER')
plt.plot(x, time['CRF'], label='CRF')

#adding the legend showing the plot
plt.xlabel('Number of Sentences')
plt.ylabel('Model Execution Time')
plt.title('Part Of Speech Models')
plt.legend()
plt.show()

In this case, PER seems to be the most effective -accuracy wise- and second most time efficient algorithm. After having performed this additional execution time analysis <b> we are reinsured in our conclusion to pick the Perceptron Model.</b> 

## Conclusions
Over the development of this lab, we have implemented 4 different POS Models, tested them with different segments of the treebank corpus and trained them with another set of segments. From those results, we have plotted their performanced based on accuracy and execution time and have selected the best performing model for this specific case, which happened to be the Perceptron Model. 

Another thing to highlight is that the HiddenMarkovModel looks really efficient timewise. It is likely that its just not in the desired set of conditions regarding the amount of data to perform correctly. 
